## Setting Envirionment

In [1]:
# ! pip install pandas
# ! pip install openai
# ! pip install langchain

In [2]:
import os
import json
import openai
import pandas as pd
import re
import time

In [3]:
with open('openai_api_key.txt', 'r') as f:
    openai.api_key = f.read().strip()
os.environ["OPENAI_API_KEY"] = openai.api_key

# os.environ.get("OPENAI_API_KEY") # Requires API key verification

## Preparing required files

In [4]:
# Load the CSV file
scn_FileName= "CAUS"
df = pd.read_csv(f'_output_scene/scene_{scn_FileName}.csv')
print(len(df))

511


In [5]:
# Calling prompts
with open('rprompt.txt', 'r') as f:
    rprompt = f.read()

with open('5qqprompt.txt', 'r') as f:
    qprompt = f.read()

## Acquiring reasoning and queries from scene list

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(model="gpt-4-0613", temperature = 0)

### First loop for reasoning and questioning from scene lists

In [7]:
# Generating reasoning and query_value from the scene description
intermediate_results = []
for i, row in df.iterrows():
    row_dict = row.to_dict()
    SceneDescription = row.iloc[1]
    
    messages = [SystemMessage(content=f"{rprompt}"), HumanMessage(content=f"{SceneDescription}")]
    reasoning = chat(messages).content

    messages = [SystemMessage(content=f"{qprompt}"), HumanMessage(content=f"{SceneDescription}")]
    query = chat(messages)
    query_value = [s.strip() for s in query.content.split(';')]

    intermediate_results.append({
        'row_dict': row_dict,
        'reasoning': reasoning,
        'query_value': query_value
    })

time.sleep(1)

### Saving intermediate file

In [9]:
# Save the intermediate_results to a JSON file
with open(f'_output_intermediate/intermediate_{scn_FileName}.json', 'w') as json_file:
    json.dump(intermediate_results, json_file)